In [1]:
import os
import glob
import keras

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from skimage import transform
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
num_final_hidden_layer = 32
num_classes = 2
batch_size = 32

checkpoint_dir = '../experiments/baseline-celeba'
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [3]:
model = Sequential()
model.add(Conv2D(
        32, kernel_size=4, strides=2, activation='relu',
        input_shape=(128, 128, 3), padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(512, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(num_final_hidden_layer, kernel_size=4))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [4]:
stopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_dir, 'checkpoint'),
    monitor='val_acc', save_best_only=True)
model.compile(optimizer=Adam(lr=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
datagen = ImageDataGenerator(rescale=1./255)
train_datagen = datagen.flow_from_directory(
        '/home/supervised-vae/data/celeba/train',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

val_datagen = datagen.flow_from_directory(
        '/home/supervised-vae/data/celeba/validation',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

test_datagen = datagen.flow_from_directory(
        '/home/supervised-vae/data/celeba/test',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

Found 182338 images belonging to 2 classes.
Found 10130 images belonging to 2 classes.
Found 10131 images belonging to 2 classes.


In [6]:
history = model.fit_generator(
    train_datagen, validation_data=val_datagen,
    callbacks=[stopping, checkpoint], epochs=100,
    steps_per_epoch=len(train_datagen),
    validation_steps=len(val_datagen))

Epoch 1/100
5699/5699 [==============================] - 3394s 596ms/step - loss: 0.1993 - acc: 0.9202 - val_loss: 0.1522 - val_acc: 0.9436
Epoch 2/100
5699/5699 [==============================] - 322s 56ms/step - loss: 0.0840 - acc: 0.9687 - val_loss: 0.1641 - val_acc: 0.9405
Epoch 3/100
5699/5699 [==============================] - 325s 57ms/step - loss: 0.0432 - acc: 0.9853 - val_loss: 0.1501 - val_acc: 0.9521
Epoch 4/100
5699/5699 [==============================] - 324s 57ms/step - loss: 0.0196 - acc: 0.9944 - val_loss: 0.1411 - val_acc: 0.9587
Epoch 5/100
5699/5699 [==============================] - 326s 57ms/step - loss: 0.0093 - acc: 0.9977 - val_loss: 0.2351 - val_acc: 0.9410
Epoch 6/100
5699/5699 [==============================] - 324s 57ms/step - loss: 0.0056 - acc: 0.9986 - val_loss: 0.1642 - val_acc: 0.9583
Epoch 7/100
5699/5699 [==============================] - 324s 57ms/step - loss: 0.0037 - acc: 0.9990 - val_loss: 0.2019 - val_acc: 0.9576


In [7]:
result = model.evaluate_generator(test_datagen, steps=len(test_datagen))
result

[0.15172429602320525, 0.9573586023509261]